# EDA Notebook

This notebook is used for exploratory data analysis

In [1]:
#imports 
import os, shutil
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score
import time
import matplotlib.pyplot as plt
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [2]:
train_images_dir = 'C:/Users/User/Documents/Flatiron/Capstone/train_data/images'
train_images_dir_0 = 'C:/Users/User/Documents/Flatiron/Capstone/train_data/images/0'
train_images_dir_1 = 'C:/Users/User/Documents/Flatiron/Capstone/train_data/images/1'

In [3]:
train_images_list_0 = [file for file in os.listdir(train_images_dir_0)
                     if (file.endswith('.jpg') or
                         file.endswith('.JPG') or
                         file.endswith('.jpeg'))]

In [4]:
train_images_list_1 = [file for file in  os.listdir(train_images_dir_1)
                     if (file.endswith('.jpg') or
                         file.endswith('.JPG') or
                         file.endswith('.jpeg'))]

In [5]:
train_images_list = train_images_list_0.copy()
train_images_list.extend(train_images_list_1)

In [6]:
print(train_images_list)

['billete_0001.jpg', 'billete_0002.jpg', 'billete_0003.jpg', 'billete_0004.jpg', 'billete_0005.jpg', 'billete_0006.jpg', 'billete_0007.jpg', 'billete_0008.jpg', 'billete_0010.jpg', 'billete_0011.jpg', 'billete_0012.jpg', 'billete_0013.jpg', 'billete_0014.jpg', 'billete_0015.jpg', 'billete_0016.jpg', 'billete_0017.jpg', 'billete_0020.jpg', 'billete_0021.jpg', 'billete_0022.jpg', 'billete_0023.jpg', 'billete_0024.jpg', 'billete_0025.jpg', 'billete_0026.jpg', 'billete_0027.jpg', 'billete_0031.jpg', 'billete_0032.jpg', 'billete_0033.jpg', 'billete_0034.jpg', 'billete_0035.jpg', 'billete_0036.jpg', 'billete_0037.jpg', 'billete_0038.jpg', 'billete_0043.jpg', 'billete_0044.jpg', 'billete_0045.jpg', 'billete_0046.jpg', 'billete_0047.jpg', 'billete_0048.jpg', 'billete_0050.jpg', 'billete_0052.jpg', 'billete_0053.jpg', 'billete_0055.jpg', 'billete_0056.jpg', 'billete_0057.jpg', 'billete_0058.jpg', 'billete_0059.jpg', 'billete_0060.jpg', 'billete_0061.jpg', 'billete_0063.jpg', 'billete_0064.jpg',

In [7]:
print('There are', len(train_images_list), 'train images')

There are 5002 train images


In [8]:
print('There are ' + str(len(train_images_list_0)) + ' images in the training set of class 0')
print('There are ' + str(len(train_images_list_1)) + ' images in the training set of class 1')


There are 1752 images in the training set of class 0
There are 3250 images in the training set of class 1


#### Image Generator Object

In [9]:
datagen = ImageDataGenerator()

### Playing with images

In [15]:
image_test = load_img('C:/Users/User/Documents/Flatiron/Capstone/train_data/images/1/knife_1179.jpg').convert('RGB')

In [16]:
image_test.getpixel

<bound method Image.getpixel of <PIL.Image.Image image mode=RGB size=1280x720 at 0x1CE39198F10>>

In [17]:
image_test_array = img_to_array(image_test)

In [18]:
image_test_array.shape

(720, 1280, 3)

In [20]:
image_test_array = image_test_array.reshape((1,) + image_test_array.shape)
image_test_array.shape

(1, 720, 1280, 3)

#### Prepping data for modeling

In [27]:
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_images_dir,batch_size=5002)

Found 5002 images belonging to 2 classes.


In [28]:
train_images, train_labels = next(train_generator)

In [29]:
train_images.shape

(5002, 256, 256, 3)

In [31]:
# Explore dataset
m_train = train_images.shape[0]
num_px = train_images.shape[1]

In [32]:
print ("Number of training samples: " + str(m_train))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))

Number of training samples: 5002
train_images shape: (5002, 256, 256, 3)
train_labels shape: (5002, 2)


#### Reshaping for normal neural network (flattening)

In [33]:
train_img_nn = train_images.reshape(train_images.shape[0], -1)

In [34]:
print(train_img_nn.shape)


(5002, 196608)


In [37]:
train_y = np.reshape(train_labels[:,0], (5002,1))
train_y.shape

(5002, 1)

In [38]:
# attempting to create a validation set using train test split
from sklearn.model_selection import train_test_split

In [40]:
X_train, X_val, y_train, y_val = train_test_split(train_img_nn, train_y, test_size=0.4)

In [41]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(3001, 196608)
(2001, 196608)
(3001, 1)
(2001, 1)


In [46]:
#train_images.reshape(train_images.shape[0], -1)
X_train_baseline = X_train.reshape(X_train.shape[0], -1)
X_val_baseline = X_val.reshape(X_val.shape[0], -1)
y_train_baseline = y_train.reshape(y_train.shape[0],-1)
y_val_baseline = y_val.reshape(y_val.shape[0],-1)
print(X_train_baseline.shape)
print(X_val_baseline.shape)
print(y_train_baseline.shape)
print(y_val_baseline.shape)

(3001, 196608)
(2001, 196608)
(3001, 1)
(2001, 1)


### Baseline

In [47]:
# Build a baseline fully connected model
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(196608,))) # 2 hidden layers
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [48]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

histoire = model.fit(X_train_baseline,
                    y_train_baseline,
                    epochs=75,
                    batch_size=32,
                     validation_data = (X_val_baseline,y_val_baseline))

Epoch 1/75
94/94 [==============================] - 7s 70ms/step - loss: 0.7370 - accuracy: 0.6108 - val_loss: 0.6796 - val_accuracy: 0.6532
Epoch 2/75
94/94 [==============================] - 7s 70ms/step - loss: 0.6741 - accuracy: 0.6475 - val_loss: 0.6675 - val_accuracy: 0.6532
Epoch 3/75
94/94 [==============================] - 7s 71ms/step - loss: 0.6649 - accuracy: 0.6475 - val_loss: 0.6599 - val_accuracy: 0.6532
Epoch 4/75
94/94 [==============================] - 6s 69ms/step - loss: 0.6591 - accuracy: 0.6475 - val_loss: 0.6549 - val_accuracy: 0.6532
Epoch 5/75
94/94 [==============================] - 6s 69ms/step - loss: 0.6554 - accuracy: 0.6475 - val_loss: 0.6517 - val_accuracy: 0.6532
Epoch 6/75
94/94 [==============================] - 7s 77ms/step - loss: 0.6531 - accuracy: 0.6475 - val_loss: 0.6497 - val_accuracy: 0.6532
Epoch 7/75
94/94 [==============================] - 8s 81ms/step - loss: 0.6516 - accuracy: 0.6475 - val_loss: 0.6484 - val_accuracy: 0.6532
Epoch 8/75
94

94/94 [==============================] - 6s 66ms/step - loss: 0.6490 - accuracy: 0.6475 - val_loss: 0.6455 - val_accuracy: 0.6532
Epoch 59/75
94/94 [==============================] - 6s 66ms/step - loss: 0.6491 - accuracy: 0.6475 - val_loss: 0.6455 - val_accuracy: 0.6532
Epoch 60/75
94/94 [==============================] - 6s 66ms/step - loss: 0.6490 - accuracy: 0.6475 - val_loss: 0.6455 - val_accuracy: 0.6532
Epoch 61/75
94/94 [==============================] - 6s 66ms/step - loss: 0.6490 - accuracy: 0.6475 - val_loss: 0.6455 - val_accuracy: 0.6532
Epoch 62/75
94/94 [==============================] - 6s 67ms/step - loss: 0.6490 - accuracy: 0.6475 - val_loss: 0.6455 - val_accuracy: 0.6532
Epoch 63/75
94/94 [==============================] - 6s 65ms/step - loss: 0.6491 - accuracy: 0.6475 - val_loss: 0.6455 - val_accuracy: 0.6532
Epoch 64/75
94/94 [==============================] - 6s 66ms/step - loss: 0.6490 - accuracy: 0.6475 - val_loss: 0.6455 - val_accuracy: 0.6532
Epoch 65/75
94/94 

In [49]:
results_train = model.evaluate(X_train_baseline, y_train_baseline)

94/94 [==============================] - 2s 16ms/step - loss: 0.6490 - accuracy: 0.6475


In [50]:
results_train

[0.6490103602409363, 0.647450864315033]

## reshaping for CNN

In [51]:
X_train_cnn, X_val_cnn, y_train_cnn, y_val_cnn = train_test_split(train_images, train_labels, test_size=0.4)

In [52]:
print(X_train_cnn.shape)
print(y_train_cnn.shape)

(3001, 256, 256, 3)
(3001, 2)


In [53]:
y_train_cnn

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [54]:
y_val_cnn = np.reshape(y_val_cnn[:,0], (len(y_val_cnn),1))
y_train_cnn = np.reshape(y_train_cnn[:,0], (len(y_train_cnn),1))

In [55]:
print(X_train_cnn.shape)
print(y_train_cnn.shape)

(3001, 256, 256, 3)
(3001, 1)


In [59]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        ))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

In [62]:
history = model.fit(X_train_cnn,
                    y_train_cnn,
                    epochs=10,
                    use_multiprocessing=True,
                    validation_data = (X_val_cnn,y_val_cnn)
                   )

Epoch 1/10
94/94 [==============================] - 429s 5s/step - loss: 0.5710 - acc: 0.6878 - val_loss: 0.6164 - val_acc: 0.6542
Epoch 2/10
94/94 [==============================] - 345s 4s/step - loss: 0.5464 - acc: 0.6968 - val_loss: 0.5650 - val_acc: 0.6897
Epoch 3/10
94/94 [==============================] - 311s 3s/step - loss: 0.5132 - acc: 0.7364 - val_loss: 0.5631 - val_acc: 0.6892
Epoch 4/10
94/94 [==============================] - 387s 4s/step - loss: 0.4962 - acc: 0.7454 - val_loss: 0.5069 - val_acc: 0.7416
Epoch 5/10
94/94 [==============================] - 402s 4s/step - loss: 0.4626 - acc: 0.7791 - val_loss: 0.4723 - val_acc: 0.7621
Epoch 6/10
94/94 [==============================] - 357s 4s/step - loss: 0.4353 - acc: 0.7941 - val_loss: 0.4458 - val_acc: 0.7841
Epoch 7/10
94/94 [==============================] - 349s 4s/step - loss: 0.4134 - acc: 0.8027 - val_loss: 0.4277 - val_acc: 0.7951
Epoch 8/10
94/94 [==============================] - 330s 4s/step - loss: 0.3774 - a

In [63]:
results_val = model.evaluate(X_val_cnn, y_val_cnn)

63/63 [==============================] - 43s 682ms/step - loss: 0.5167 - acc: 0.7601


In [64]:
results_val

[0.5166699290275574, 0.7601199150085449]

### Test set creation

In [65]:
images_test_dir = 'C:/Users/User/Documents/Flatiron/Capstone/Test_data'


In [66]:
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        images_test_dir,
    target_size=(640, 360),
batch_size=32)

Found 857 images belonging to 2 classes.


In [67]:
test_images, test_labels = next(test_generator)

In [68]:
test_img = test_images.reshape(test_images.shape[0], -1)

In [69]:
test_y = np.reshape(test_labels[:,0], (32,1))
#test_y